In [83]:
import argparse
import requests
from colored import bg, attr
import os
import textwrap
import itertools
from pprint import pprint
import string

In [4]:
SCRYFALL_API = "https://api.scryfall.com/cards/search"

In [5]:
def run_query(query):
    return requests.get(SCRYFALL_API, params = {'q': query})

In [6]:
def parse_json(response):
    return response.json()

In [7]:
def print_warnings(json_reponse):
    try:
        for warn in json_reponse['warnings']:
            print(' WARN: {}'.format(warn))
    except KeyError:
        pass

In [8]:
def print_total_number_of_cards(json_response):
    print(' Total Cards: {}'.format(json_response['total_cards']))

In [9]:
def try_get(card, item):
    try:
        return card[item]
    except KeyError:
        return ''

In [10]:
def main(query, cards_to_match, text=False):
    print("Query Length: {}".format(len(query)))
    response = run_query(query)
    json_response = parse_json(response)
    print_warnings(json_response)
    print_total_number_of_cards(json_response)
    print_page(json_response, cards_to_match, text)

In [11]:
def print_page(json_response, cards_to_match, text=False):
    for card in json_response['data']:
        print_card(card, cards_to_match, text)

In [100]:
text_width=50

def print_card(card, cards_to_match, text=False):
    #pprint(card)
    artist = card['artist']
    set_name = try_get(card, 'set')
    power = try_get(card, 'power')
    toughness = try_get(card, 'toughness')
    name = card['name']
    type_line = card['type_line']
    rarity = try_get(card, 'rarity')
    oracle = textwrap.wrap(try_get(card, 'oracle_text').replace('\n', ''), width=text_width)
    mana_cost = try_get(card, 'mana_cost').replace('{','').replace('}','')
    identity = ''.join(try_get(card, 'color_identity'))
    flavor = textwrap.wrap(try_get(card, 'flavor_text').replace('\n', ''), width=text_width)
    cmc = int(card['cmc'])
    
    output = f'{name:<35.34}{identity:<6}{artist:<20.19}{set_name:<4.3}{rarity[0]:<2.1}{cmc:<3}{mana_cost:<10}{type_line:<40.39}{power}/{toughness}'
    
    if name in cards_to_match:
        output = bg('deep_sky_blue_3a') + output + attr('reset')
    print(output)
    
    if text:
        for o_text, f_text in itertools.zip_longest(oracle, flavor, fillvalue=''):
            print(f"    {o_text:<50.50}    {f_text:<50.50}")
        print()

In [101]:
def compare_names(name_1, name_2):
    n1_set = set(name_1.lower().replace(' ',''))
    n2_set = set(name_2.lower().replace(' ',''))
    
    in_both = n1_set & n2_set
    print(f"   in both: {''.join(sorted(in_both))}")
          
    in_neither = set(string.ascii_lowercase)
    in_neither -= n1_set
    in_neither -= n2_set
    
    print(f"in neither: {''.join(sorted(in_neither))}")

In [102]:
compare_names('Torment of Hailfire','Laboratory Brute')

   in both: aelort
in neither: cdgjkpqsvwxz


In [130]:
compare_names('Yeong-Hao Han','Grzegorz Rutkowski')

   in both: ego
in neither: bcdfjlmpqvx


In [138]:
compare_names('Creature — Zombie Horror','Sorcery')

   in both: ceor
in neither: dfgjklnpqvwx


In [103]:
main('(Laboratory Brute) or (Torment of Hailfire)',
     [],
     text=True)

Query Length: 43
 Total Cards: 2
Laboratory Brute                   U     Yeong-Hao Han       emn c 4  3U        Creature — Zombie Horror                3/3
    When Laboratory Brute enters the battlefield, put     It keeps intruders out and experiments in.        
    the top four cards of your library into your                                                            
    graveyard.                                                                                              

Torment of Hailfire                B     Grzegorz Rutkowski  hou r 2  XBB       Sorcery                                 /
    Repeat the following process X times. Each            "Your God-Pharaoh has returned."                  
    opponent loses 3 life unless that player                                                                
    sacrifices a nonland permanent or discards a card.                                                      



In [172]:
main('"or" -s ft:ou -t:n t:o a:g',
     ['Torment of Hailfire','Laboratory Brute'],
     text=False)

Query Length: 26
 Total Cards: 17
By Force                           R     Magali Villeneuve   akh u 1  XR        Sorcery                                 /
Cryptborn Horror                   BR    Richard Wright      rtr r 3  1B/RB/R   Creature — Horror                       0/0
Fetid Horror                       B     Gary Leach          mir c 4  3B        Creature — Shade Horror                 1/2
Goblin Lore                        R     D. Alexander Gregor 10e u 2  1R        Sorcery                                 /
Inkfathom Infiltrator              BU    Jesper Ejsing       shm u 2  U/BU/B    Creature — Merfolk Rogue                2/1
Kjeldoran Gargoyle                 W     Marcelo Vignali     csp u 6  5W        Creature — Gargoyle                     3/3
Laboratory Brute                   U     Yeong-Hao Han       emn c 4  3U        Creature — Zombie Horror                3/3
Lizard Warrior                     R     Roger Raupp         cns c 4  3R        Creature — Lizard Warr